<a href="https://www.kaggle.com/code/babe8901/object-detection-using-deep-learning?scriptVersionId=107759138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
! pip install numpy
! pip install pandas
! pip install matplotlib
! pip install opencv-python
! pip install tqdm
! pip install sklearn
! pip install pyyaml
! pip install torch

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 
import os 
import shutil 
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split
import yaml
import torch

In [ ]:
annotations = pd.read_csv('../input/car-object-detection/data/train_solution_bounding_boxes (1).csv')
annotations.head()

# Looking at some examples from training set

In [ ]:
base_path = '../input/car-object-detection/data/training_images'
fig, ax = plt.subplots(figsize=(16,16), nrows=4, ncols=4)
nrows=4
ncols=4
indices = np.random.randint(low=0, high=len(annotations), size=(nrows, ncols))
for i in range(nrows):
    for j in range(ncols):
        idx = indices[i,j]
        row = annotations.loc[idx]
        start = (int(row['xmin']), int(row['ymin']))
        end = (int(row['xmax']), int(row['ymax']))
        img_path = os.path.join(base_path, row['image'])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.rectangle(img, start, end, color=(0,255,0), thickness=2)
        ax[i,j].axis('off')
        ax[i,j].imshow(img)
plt.tight_layout()
plt.show()

# Making annotations compatible with YOLO

In [ ]:
annotations['xcenter'] = (annotations['xmin']+annotations['xmax'])/2
annotations['ycenter'] = (annotations['ymin']+annotations['ymax'])/2
annotations['width'] = (annotations['xmax'] - annotations['xmin'])
annotations['height'] = (annotations['ymax'] - annotations['ymin'])

In [ ]:
h,w,c = img.shape

In [ ]:
annotations['xcenter'] = annotations['xcenter']/w
annotations['width'] = annotations['width']/w
annotations['ycenter'] = annotations['ycenter']/h
annotations['height'] = annotations['height']/h
annotations['image'] = annotations['image'].apply(lambda x: os.path.join(base_path,x))
annotations = annotations[['image', 'xcenter','ycenter','width','height']]

In [ ]:
annotations.head()

# Creating train, test and val folders

In [ ]:
train_path = os.path.join('Data','train')
test_path = os.path.join('Data','test')
val_path = os.path.join('Data','val')

os.makedirs(train_path, exist_ok=True)
print('Made folder for train images')
os.makedirs(val_path, exist_ok=True)
print('Made folder for validation images')
os.makedirs(test_path, exist_ok=True)
print('Made folder for test images')

In [ ]:
test_size = int(0.1 * len(annotations))
train_df, test_df = train_test_split(annotations, test_size=test_size)
train_df, val_df = train_test_split(train_df, test_size=test_size)

In [ ]:
data = {
    'train':{
        'df':train_df,
        'path':train_path
    },
    'val':{
        'df':val_df,
        'path':val_path
    },
    'test':{
        'df':test_df,
        'path':test_path
    }
}

# Moving the files to train, val and test folders

In [ ]:
for subset, subset_data in data.items():
    df = subset_data['df']
    dst_path = subset_data['path']
    for idx, row in tqdm(df.iterrows()):
        src = row['image']
        img_id = src.split(os.path.sep)[-1].split('.')[0]
        extension = src.split(os.path.sep)[-1].split('.')[1]
        img_dst = os.path.join(dst_path, f'{img_id}.{extension}')
        shutil.copy2(src, img_dst)
        annotation_text = f"0 {row['xcenter']} {row['ycenter']} {row['width']} {row['height']}"
        with open(os.path.join(dst_path, f'{img_id}.txt'), 'w+') as f:
            f.write(annotation_text)
            f.write('\n')
    print(f'Done moving files for {subset} set')

In [ ]:
config = {
    'names':['Car'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path),
    'test':os.path.abspath(test_path)
}

with open('data.yaml', 'w+') as f:
    yaml.dump(config, f, default_flow_style=False)

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
! pip install -r yolov5/requirements.txt

In [ ]:
! wandb disabled

In [ ]:
! python yolov5/train.py --data data.yaml --img 640 --batch 16 --epochs 20 --weights yolov5/yolov5s.pt

In [ ]:
best_model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5/runs/train/exp/weights/best.pt')

# Inference on test images

In [ ]:
test_base_path = '../input/car-object-detection/data/testing_images'
test_img = os.listdir(test_base_path)
test_img = list(map(lambda x: os.path.join(test_base_path, x), test_img))
test_images = np.random.choice(test_img, size=(4,4))

In [ ]:
fig, ax = plt.subplots(figsize=(16,16), nrows=4, ncols=4)
for i in range(4):
    for j in range(4):
        image = cv2.imread(test_images[i,j])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = best_model(image)
        ax[i,j].axis('off')
        ax[i,j].imshow(np.squeeze(results.render()))
plt.show()